# Paper Reproducing
In this part we will attempt to reproduce the correlation results from the original paper. This is done both to provide a sanity check, but also to create the general functions we will use later to calcuate the correlation on the data from the models

In [1]:
import pandas as pd
import netCDF4 as nc
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy import interpolate
import scipy
import tqdm
import matplotlib.colors as mcolors
from helpers import process_h as helper
from helpers import visualize_h as vis_helper


In [2]:
data_relative_path = os.path.join('..', 'Data')
precipitation_path = os.path.join(data_relative_path,'CMIP6','Precipitation')
WTIO_path = os.path.join(data_relative_path,'CMIP6','WTIO')
SEIO_path = os.path.join(data_relative_path,'CMIP6','SEIO')
CHIRPS_path = os.path.join(data_relative_path,'CHIRPS2')

# Constants 

In [3]:
df_dmi = pd.read_csv(f"{data_relative_path}/PSL/DMI.csv")
df_wtio = pd.read_csv(f"{data_relative_path}/PSL/WTIO.csv")
df_seio = pd.read_csv(f"{data_relative_path}/PSL/SEIO.csv")

bounds_lat = [20, 40]
bounds_lon = [20, 50]
boundaries = bounds_lon + bounds_lat

In [4]:
df_dmi.set_index('Year', inplace=True)
df_wtio.set_index('Year', inplace=True)
df_seio.set_index('Year', inplace=True)

In [5]:
df_dmi = df_dmi.query('Year >= 1981 and Year <= 2010')
df_wtio = df_wtio.query('Year >= 1981 and Year <= 2010')
df_seio = df_seio.query('Year >= 1981 and Year <= 2010')

In [6]:
dmi_oct = df_dmi['10']
wtio_oct = df_wtio['10']
seio_oct = df_seio['10']
obs_iod = pd.concat([dmi_oct, wtio_oct, seio_oct], axis=1, keys=['DMI', 'WTIO', 'SEIO'])

In [7]:
obs_iod.plot()
plt.legend(['DMI', 'WTIO', 'SEIO'])
obs_iod.corr()['DMI']['WTIO']

Next, we will plot the correlation between the DMI and the precipitation data.

In [8]:
dmi_oct.values

In [10]:
# Attempt

PATH = os.path.join('..', 'Data', 'CHIRPS2')

def read_data(file_path):
    data = nc.Dataset(file_path)
    return data

ds = read_data(os.path.join(PATH, 'chirps-v2.0.monthly.nc'))

time_var = ds.variables["time"][:]
fig, ax = plt.subplots(1, 3, figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})
for mod_ind in range(9, 12):
    october_indices = [i for i, t in enumerate(time_var) if (i + 1) % mod_ind == 0 ]
    october_indices = october_indices[0:30]

    lat_var = ds.variables["latitude"][:]
    lon_var = ds.variables["longitude"][:]

    lat_indices = np.where((lat_var >= 20) & (lat_var <= 40))[0]
    lon_indices = np.where((lon_var >= 20) & (lon_var <= 50))[0]

    precip_data = ds.variables["precip"]

    october_data = precip_data[october_indices, :, :]
    october_lat_lon_data = october_data[:, lat_indices, :][:, :, lon_indices]
    corr_map = helper.calc_dmi_precp_corr_loop(dmi_oct.values, october_lat_lon_data)
    alpha = 0.05
    corr_map_tested = np.where(helper.calc_corr_t_test(corr_map) > alpha, 0, corr_map)
    vis_helper.plot_corr_heatmap(ax[mod_ind % 3 - 1], corr_map, boundaries, f'Corr of DMI and Precip, {mod_ind} month', f'Corr. between DMI and CHIRPS, {mod_ind} month', 0)

plt.tight_layout()
plt.show()

In [ ]:
chirps_data = nc.Dataset(f"{CHIRPS_path}/chirps_octobers_middle_east_1981_2010.nc") # CHIRPS_Monthly_precipitation

chirps_precip_data = chirps_data['precip'][:]


corr_map = helper.calc_dmi_precp_corr_loop(dmi_oct.values, chirps_precip_data)

# corr_map = calc_dmi_precp_corr_vec(dmi_oct.values, chirps_precip_data)

In [ ]:
alpha = 0.05
corr_map_tested = np.where(helper.calc_corr_t_test(corr_map) > alpha, 0, corr_map)

In [ ]:
############# PLOT MAPS 4 #############
# Create a 2D meshgrid for the latitude and longitude data
lon = chirps_data['longitude'][:]
lat = chirps_data['latitude'][:]
lon_2d, lat_2d = np.meshgrid(lon, lat)
fig = plt.figure()
ax = plt.axes()

#map = Basemap(projection='cyl', llcrnrlon=33.,llcrnrlat=29.,urcrnrlon=37.,urcrnrlat=34., resolution='i', ax=ax)
map = Basemap(projection='merc',llcrnrlon=20,llcrnrlat=20,urcrnrlon=50,urcrnrlat=40,resolution='i') # projection, lat/lon extents and resolution of polygons to draw
# resolutions: c - crude, l - low, i - intermediate, h - high, f - full
map.drawcoastlines()
map.drawstates()
map.drawlsmask(land_color='Linen', ocean_color='#CCFFFF') # can use HTML names or codes for colors
map.drawcounties() # you can even add counties (and other shapefiles!)

#map.pcolormesh(lon_2d, lat_2d, pressureS[t,:,:], latlon=True, cmap='coolwarm') #Can change variables and index for time - also in line 48
cmap = 'coolwarm'
#vmin = 100000
#vmax = 102000

#cs = map.contour(lon_2d, lat_2d, pressureS[t, :, :], latlon=True, cmap=cmap)#, vmin=vmin, vmax=vmax) #lines
cs = map.contourf(lon_2d, lat_2d, corr_map, latlon=True, cmap=cmap)#, vmin=vmin, vmax=vmax) #color lines
#cs = map.pcolormesh(lon_2d, lat_2d, pressureS[t, :, :], latlon=True, cmap=cmap)#, vmin=vmin, vmax=vmax)
#Diverging Colormaps: 'coolwarm', 'RdBu_r', 'seismic', 'viridis', 'plasma', 'inferno', 'cividis', 'magma', 'tab10', ''
# Adding pressure isobar lines
#levels = np.arange(100000, 102000, 100) # define pressure levels
#contour = map.contour(lon_2d, lat_2d, pressureS[t, :, :], levels=levels, latlon=True, colors='k', linewidths=1)

# Adding labels to contour lines
#plt.clabel(cs, inline=1, fontsize=10, fmt='%1.0f')

map.colorbar()
#cbar = map.colorbar(cs, location='bottom', pad="5%")
#cbar.set_label('Pressure (Pa)')
parallels = np.arange(min(lat),max(lat),1.) # make latitude lines ever 5 degrees from 30N-50N
meridians = np.arange(min(lon),max(lon),1.) # make longitude lines every 5 degrees from 95W to 70W
map.drawparallels(parallels,labels=[1,0,0,0],fontsize=8, dashes=[1, 3], color='lightgray', linewidth=0.5)
map.drawmeridians(meridians,labels=[0,0,0,1],fontsize=8, dashes=[1, 3], color='lightgray', linewidth=0.5)

#plt.xlabel('Longitude')
#plt.ylabel('Latitude')
ax.set_ylabel('Latitude', loc='center' , labelpad=20.0)
ax.set_xlabel('Longitude', loc='center', labelpad=13.0)
plt.title('Correlation between DMI and CHIRPS') #Change title per variable

plt.show()

In [13]:
fig7, ax7 = plt.subplots(1, 1, figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})
vis_helper.plot_corr_heatmap(ax7, corr_map, boundaries, 'Correlation between DMI and Precipitation', 'Corr. between DMI and CHIRPS', 0)

In [15]:
corr_map_tested

In [ ]:
helper.calc_corr_t_test(corr_map)

In [37]:
corr_map

In [ ]:
expanded_array = dmi_oct.values[:, np.newaxis, np.newaxis]

# Step 3: Use broadcasting to create the desired shape (30, 400, 600)
result_array = np.tile(expanded_array, (1, 400, 600))


In [ ]:
scipy.stats.pearsonr(chirps_precip_data,result_array)

In [16]:
chirps_precip_data

In [21]:
chirps_data.variables['precip']

In [22]:
chirps_data.variables['time']